<a href="https://colab.research.google.com/github/iued-uni-heidelberg/DAAD-Training-2021/blob/main/Terminologieextraktion3WordsAndMWEs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Terminology extraction experiments

## Improving and running terminology extraction script on different corpora



In [1]:
# notebook Terminologieextraktion3WordsAndMWEs.ipynb is a production stage version: all debugging printouts have been removed; optimization for speed

In [2]:
# ArmenianWP
# !wget https://heibox.uni-heidelberg.de/f/206d85a0270943e4b87b/?dl=1
# renaming file
# !mv index.html?dl=1 WPhy_vert.txt

# Constitution Republic of Armenia
# !wget https://heibox.uni-heidelberg.de/f/bf54977b17604ec592cd/?dl=1
# renaming file
# !mv index.html?dl=1 Const_RA_l.txt

# Grundgesetz
# !wget https://heibox.uni-heidelberg.de/f/d6c5b31edc84422d9e14/?dl=1
# renaming file
# !mv index.html?dl=1 GG_l.txt

# Origin of the species
# !wget https://heibox.uni-heidelberg.de/f/befc6dbe718b4a37ba74/?dl=1
# renaming file
# !mv index.html?dl=1 OOOS_l.txt

# Europarl DE
# !wget https://heibox.uni-heidelberg.de/f/3ba6122e744e4b7f9c14/?dl=1
# renaming file
# !mv index.html?dl=1 EP_DE_l.txt

# 23.10.2021 part
# German legal corpus, lemmatized in a zip archive
!wget https://heibox.uni-heidelberg.de/f/fd96c36723b741d4a972/?dl=1
# renaming file
!mv index.html?dl=1 BGH-utf8-lem.zip


--2021-10-24 10:41:30--  https://heibox.uni-heidelberg.de/f/fd96c36723b741d4a972/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/b4d03479-8c5c-43f5-bf87-9c68a6f111aa/output-utf8-lem.zip [following]
--2021-10-24 10:41:31--  https://heibox.uni-heidelberg.de/seafhttp/files/b4d03479-8c5c-43f5-bf87-9c68a6f111aa/output-utf8-lem.zip
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 673144348 (642M) [application/zip]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>] 641.96M  14.9MB/s    in 43s     

2021-10-24 10:42:14 (14.9 MB/s) - ‘index.html?dl=1’ saved [673144348/673144348]



In [3]:
!unzip BGH-utf8-lem.zip
!rm BGH-utf8-lem.zip

Archive:  BGH-utf8-lem.zip
  inflating: output-utf8-lem.txt     


In [4]:
!mv output-utf8-lem.txt BGHlem.txt

In [5]:
!head --lines=100000 BGHlem.txt >BGHlem1k.txt

In [6]:
# OPTIONAL -- just to know how big is our corpus, on a long corpus it can take a lot of time...
# word counts: BGHlem.txt should be ~221 M lines long (vert lemmatized format; so one line is one word)
!wc BGHlem1k.txt
!wc BGHlem.txt

 100000  299903 1673786 BGHlem1k.txt
 220915764  662340855 3684207041 BGHlem.txt


In [7]:
# OPTIONAL -- just to demonstrate the algorithm of term selection
# debugging and explaining the algorithm for Stage 1:
LLCandidates = []
LWords = ['Revisionsverfahren', 'entstanden', 'notwendig', 'Auslage']
for klen in range(len(LWords)): # lengths of candidate lists
    klength = klen+1 # true length: for 0 it is le = 1
    print(f'klen:{klength};')
    for i in range(len(LWords) - klen): # positions where candidates start
        print(f'i:{i};')
        LCandidate = LWords[i:i+klength]
        LLCandidates.append(LCandidate)
        print(LCandidate)

        # print(LWords[i])

for LEl in LLCandidates:
    # for el in LEl:
    print(LEl)

klen:1;
i:0;
['Revisionsverfahren']
i:1;
['entstanden']
i:2;
['notwendig']
i:3;
['Auslage']
klen:2;
i:0;
['Revisionsverfahren', 'entstanden']
i:1;
['entstanden', 'notwendig']
i:2;
['notwendig', 'Auslage']
klen:3;
i:0;
['Revisionsverfahren', 'entstanden', 'notwendig']
i:1;
['entstanden', 'notwendig', 'Auslage']
klen:4;
i:0;
['Revisionsverfahren', 'entstanden', 'notwendig', 'Auslage']
['Revisionsverfahren']
['entstanden']
['notwendig']
['Auslage']
['Revisionsverfahren', 'entstanden']
['entstanden', 'notwendig']
['notwendig', 'Auslage']
['Revisionsverfahren', 'entstanden', 'notwendig']
['entstanden', 'notwendig', 'Auslage']
['Revisionsverfahren', 'entstanden', 'notwendig', 'Auslage']


In [8]:
#Terminologieextraktion
import os, re, sys
class clProcCorpus(object):
    ''' we will read a text file and return a dictionary
    this will be done on the line by line basis
    The dictionary can be sorted later...
    '''
    # this class is processing a corpus

    def __init__(self, FileIN):
        self.DictFrq = {}
        self.processCorpus(FileIN)


    def procCorpSelectField4FrqDic(self, IFieldNumber, L2PossibleTerm):
        '''
        this functions takes a 2D list (just one possible term at a time, with all fields) and returns a string which represents the term -- either as inflected words or lemmas
        '''
        LSelectedField = []
        for LFields in L2PossibleTerm:
            LSelectedField.append(LFields[IFieldNumber])
        
        SSelectedField = ' '.join(LSelectedField)
        return SSelectedField

        


    def processCorpus(self, FileIN):
        # here we consider a larger MWE, which has been collected, e.g., 
        # Wahl Schluckebier Nachschlagewerk; or:
        # graphical user interface
        # in the default version only the longest string is preserved;
        # we try to split it into meaningful smaller units and preserve them as well:
        # Wahl Schluckebier
        # graphical user ; user interface ; interface
        # PoS restrictions apply: 
        # - Adj cannot be at the end; function words (Prepositions, articles, etc.) cannot be on either edge.
        # so we keep the list of tuples: and extract smaller MWEs from the larger strings using PoS restrictions

        # Stage 0: collecting the longest string, with allowed PoS codes, e.g., the longest contiuous string of N, Adj, Prep, Gen. articles
        LLTerm = [] # this is the List of Lists: list of words (with their fields), representing a sequence of allowed PoS codes
        n = 0 # count lines processed (monitoring progress over the corpus)
        for Line in FileIN:
            n+=1
            if n%1000000 == 0: print(n) # print every 1 Millonth line number
            Line = Line.strip()
            LLine = re.split('\t', Line)
            
            try:
                Word = LLine[0]
                PoS = LLine[1]
                Lemma = LLine[2]
            except:
                Word = ""
                PoS = ""
                Lemma = ""
            
            #Select the Tags for your langauge
            # update: this is now done in two stages: 
            # Stage 1: a set of any PoS codes which may contain candidates
            # Stage 2: restrictions on sequences (e.g., what is not allowed at the end, start...)
            # 
            # CHANGE HERE FOR YOUR LANGUAGE: stage 1
            #if re.match('N.*', PoS) or re.match('A.*', PoS): #Arm
            #if re.match('N.*', PoS) or re.match('J.*', PoS): #EN
            if re.match('N.*', PoS) or re.match('ADJ.*', PoS): #DE
                LLTerm.append(LLine) # all the field as a list, to form the list of lists

                # Terms as Words or Lemmas -- MOVED TO Stage 2
                # to compare with the gold standard do we need words, or do we need to generate words from lemas?
                # LTerm.append(Lemma)
                # here we keep all the fields, because sub-sections of the longer MWE would also be analysed using PoS codes

            else: # end of the 'candidate collection'
                ### changed to an algorithm based on the data format: list of lists

                # Stage 1: we generate candidate sub-n-grams using the longest allowed sequence, starting from single words to the longest one (the whole sequence)
                L3Candidates = [] # this will be the list of all MWE candidates (sub-sequences)
                IMaxMWE = len(LLTerm) # this is the longest MWE we can get from LLTerm: the length of the whole sequence
                for klen in range(IMaxMWE): # lengths of candidate lists: starting from 0 till, but not including the max length
                    klength = klen+1 # converting to true length: from length 1 till, and including the max length (we need this for slicing)
                    # print(f'klen:{klength};')
                    for i in range(IMaxMWE - klen): # positions where candidates start: position is from 0... up to (depending on length)
                        # print(f'i:{i};')
                        L2Candidate = LLTerm[i:i+klength] # L2 stands fro 'list of lists' (because we preserve 'word', 'lemma', 'pos' as a list for each item)
                        L3Candidates.append(L2Candidate) # L3 is 'list of lists of lists' (here we collect all candidate sub-sequences from the longest L2 sequence)

                # Stage 2: filtering by part-of-speech configurations: what is impossible, e.g., Adjectives can only appear in the beginning!
                # CHANGE HERE FOR YOUR LANGUAGE: stage 2
                L3PossibleTerms = [] # filtered lists of terms
                for L2Candidate in L3Candidates: # for each candidate configuration
                    # e.g., if Adj is the last element (at the end) -- do not add
                    # ADD MORE CONFIGURATIONS, eg, Gen case articles, prepositions
                    # FileDebug.write(str(L2Candidate) + '\n') # TEMPORARY -  DEBUGGING
                    # Word = L2Candidate[WNumber][0]; PoS = L2Candidate[WNumber][1]; Lemma = L2Candidate[WNumber][2]
                    # CONF 1: remove adjectives at the end: ListOfFields4LastWord = L2Candidate[-1] 
                    if re.match('ADJ.*', L2Candidate[-1][1]): 
                        # FileDebug.write('\t Removed:' + str(L2Candidate) + '\n')
                        continue
                    # ADD MORE CONFIGURATIONS HERE, e.g, Gen. case articles...
                    else: # when all filters passed -- add all fields; frq list of terms will be represented by the Word or Lemma field in the end
                        L3PossibleTerms.append(L2Candidate)
                # here we have L3PossibleTerms ready; con
                # FileDebug.write('---\n' + str(L3PossibleTerms) + '\n---\n')

                # going over all elements in the list of kept PossibleTerms, converting to each to a string and adding each one to the frequency dictionary
                
                for L2PossibleTerm in L3PossibleTerms:
                    # CHOOSE SECOND ARGUMENT TO COLLECT: Word = 0; PoS = 1; Lemma = 2
                    SPossibleTerm = self.procCorpSelectField4FrqDic(0, L2PossibleTerm)

                    try:
                        # self.DictFrq[STerm] += 1
                        self.DictFrq[SPossibleTerm] += 1
                    except:
                        # self.DictFrq[STerm] = 1
                        self.DictFrq[SPossibleTerm] = 1  

                LLTerm = [] # now we clear the list represening allowed PoS sequence, and start over again (we are in the else, we encountered end of allowed PoS codes)
                     
        return



In [9]:
# FileDebug = open('BGH_debug.txt', 'w')
# TEST:
# FileIN = open('BGHlem1k.txt', 'r')
# PRODUCTION:
FileIN = open('BGHlem.txt', 'r')


FileOut1w = open('BGH_term1w.txt', 'w')
FileOutMWE = open('BGH_termMWE.txt', 'w')

In [10]:
# save the frequency dictionary into file, by decreasing frequencies
# FileOutput.write( str( DictionaryFrq ) + '\n' )

OCorpus = clProcCorpus(FileIN)
DictionaryFrq = OCorpus.DictFrq


for Word, Frq in sorted( DictionaryFrq.items() , key=lambda x: x[1], reverse=True):
    if re.search(' ', Word):
        FileOutMWE.write(Word + '\t' + str(Frq) + '\n')
    else:
        FileOut1w.write(Word + '\t' + str(Frq) + '\n')

FileOutMWE.flush()
FileOutMWE.close()
FileOut1w.flush()
FileOut1w.close()


1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
68000000
69000000
70000000
71000000
72000000
73000000
74000000
75000000
76000000
77000000
78000000
79000000
80000000
81000000
82000000
83000000
84000000
85000000
86000000
87000000
88000000
89000000
90000000
91000000
92000000
93000000
94000000
95000000
96000000
97000000
98000000
99000000
100000000
101000000
102000000
103000000
104000000
105000000
106000000
107000000
108000000
109000000
110000000
11100000

In [11]:
!zip BGH_term1w.zip BGH_term1w.txt
!zip BGH_termMWE.zip BGH_termMWE.txt

  adding: BGH_term1w.txt (deflated 63%)
  adding: BGH_termMWE.txt (deflated 74%)


In [12]:
# optional: how large are our output files for MWEs and single words?
!wc BGH_term1w.txt
!wc BGH_termMWE.txt

 476230  952039 8954371 BGH_term1w.txt
 2448949  8875996 78612600 BGH_termMWE.txt
